# COURSE: A deep understanding of deep learning
## SECTION: ANNs
### LECTURE: Model depth vs. breadth
#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/deeplearning_x/?couponCode=202305

In [ ]:
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision

# Import and organize the data

In [ ]:
MNIST = torchvision.datasets.MNIST(".", download=True)
data = MNIST.data
labels = MNIST.targets

# Randomly drop samples to shrink the size to 20,000
np.random.seed(42)  # Set random seed for reproducibility
indices = np.random.choice(len(data), size=20000, replace=False)
data = data[:20000]
labels = labels[:20000]

# Reshape data to 2D array
data = data.reshape(data.shape[0], -1)
# Reshape labels to 2D array
labels = labels.reshape(labels.shape[0], -1)

data_norm = data / torch.max(data)

In [ ]:
# Step 1: convert to tensor
dataT = data_norm.clone().detach().float()
labelsT = labels.clone().detach().long()  # long = int64

# Step 2: use scikitlearn to split the data
train_data, test_data, train_labels, test_labels = train_test_split(
    dataT, labelsT, test_size=.1)

# Step 3: convert into PyTorch Datasets
train_data = TensorDataset(train_data, train_labels)
test_data = TensorDataset(test_data, test_labels)

# Step 4: translate into dataloader objects
batchsize = 32
train_loader_norm = DataLoader(train_data,
                               batch_size=batchsize,
                               shuffle=True,
                               drop_last=True)
test_loader_norm = DataLoader(test_data,
                              batch_size=test_data.tensors[0].shape[0])

In [ ]:
print(
    f"Training data NORM range from {torch.min(train_data.tensors[0])} to {torch.max(train_data.tensors[0])}"
)
print(
    f"Test data NORM range from {torch.min(test_data.tensors[0])} to {torch.max(test_data.tensors[0])}"
)

# Construct and sanity-check the model

In [ ]:
# create a class for the model
def createTheMNISTNet(num_hidden_layers, num_hidden_units):

    class mnistNet(nn.Module):

        def __init__(self):
            super().__init__()

            ### input layer
            self.input = nn.Linear(784, num_hidden_units)

            ### hidden layer
            self.fc_layers = [
                nn.Linear(num_hidden_units, num_hidden_units)
                for i in range(0, num_hidden_layers)
            ]

            ### output layer
            self.output = nn.Linear(num_hidden_units, 10)

        # forward pass
        def forward(self, x):
            x = F.relu(self.input(x))
            for hidden_layer in self.fc_layers:
                x = F.relu(hidden_layer(x))
            return self.output(x)

    # create the model instance
    net = mnistNet()

    # loss function
    lossfun = nn.CrossEntropyLoss()

    # optimizer
    optimizer = torch.optim.SGD(net.parameters(), lr=.01)

    return net, lossfun, optimizer

# Create a function that trains the model

In [ ]:
# a function that trains the model


def function2trainTheModel(train_loader,
                           test_loader,
                           numepochs,
                           num_hidden_layers=1,
                           num_hidden_units=50):

    # create a new model
    net, lossfun, optimizer = createTheMNISTNet(num_hidden_layers,
                                                num_hidden_units)

    # initialize losses
    losses = torch.zeros(numepochs)
    trainAcc = []
    testAcc = []

    # loop over epochs
    for epochi in range(numepochs):

        # loop over training data batches
        batchAcc = []
        batchLoss = []
        for X, y in train_loader:
            y = torch.flatten(y)

            # forward pass and loss
            yHat = net(X)
            loss = lossfun(yHat, y)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # loss from this batch
            batchLoss.append(loss.item())

            # compute accuracy
            matches = torch.argmax(yHat, axis=1) == y  # booleans (false/true)
            matchesNumeric = matches.float()  # convert to numbers (0/1)
            accuracyPct = 100 * torch.mean(matchesNumeric)  # average and x100
            batchAcc.append(accuracyPct)  # add to list of accuracies
        # end of batch loop...

        # now that we've trained through the batches, get their average training accuracy
        trainAcc.append(np.mean(batchAcc))

        # and get average losses across the batches
        losses[epochi] = np.mean(batchLoss)

        # test accuracy
        X, y = next(iter(test_loader))  # extract X,y from test dataloader
        with torch.no_grad():
            y = torch.flatten(y)
            yHat = net(X)

        # compare the following really long line of code to the training accuracy lines
        testAcc.append(100 * torch.mean(
            (torch.argmax(yHat, axis=1) == y).float()))

    # end epochs

    # function output
    return trainAcc, testAcc, losses, net


In [ ]:
# number of epochs
numepochs = 60
accuracy_experiments = {}

for num_layers in range(1, 4):
    accuracy_experiments[num_layers] = {}
    for num_units in range(50, 251, 50):
        print(
            f"Training with {num_layers} hidden layers and {num_units} hidden units"
        )
        train_accuracy_experiment, test_accuracy_experiment, losses_experiment, net_experiment = function2trainTheModel(
            train_loader_norm, test_loader_norm, numepochs, num_layers,
            num_units)
        accuracy_experiments[num_layers][num_units] = {
            "train_accuracy_experiment": train_accuracy_experiment,
            "test_accuracy_experiment": test_accuracy_experiment
        }


In [ ]:
accuracy_experiments

In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(16, 25))

for i, num_units in zip(range(0, 5), range(50, 251, 50)):
    ax[i][0].plot(
        accuracy_experiments[1][num_units]["train_accuracy_experiment"],
        label='1')
    ax[i][0].plot(
        accuracy_experiments[2][num_units]["train_accuracy_experiment"],
        label='2')
    ax[i][0].plot(
        accuracy_experiments[3][num_units]["train_accuracy_experiment"],
        label='3')
    ax[i][0].set_xlabel('Number of hidden units')
    ax[i][0].set_ylabel('Accuracy (%)')
    ax[i][0].set_ylim([0, 100])
    ax[i][0].set_title('Train')
    ax[i][0].legend()

    ax[i][1].plot(
        accuracy_experiments[1][num_units]["test_accuracy_experiment"],
        label='1')
    ax[i][1].plot(
        accuracy_experiments[2][num_units]["test_accuracy_experiment"],
        label='2')
    ax[i][1].plot(
        accuracy_experiments[3][num_units]["test_accuracy_experiment"],
        label='3')
    ax[i][1].set_xlabel('Number of hidden units')
    ax[i][1].set_ylabel('Accuracy (%)')
    ax[i][1].set_ylim([0, 100])
    ax[i][1].set_title("Test")
    ax[i][1].legend()

plt.show()